In [1]:
%load_ext autoreload
%autoreload 2

from context import DATADIR, RAWDATADIR
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

data = pd.read_csv(RAWDATADIR / 'public.csv')
train = data[~data['North'].isna()]
test = data[data['North'].isna()]

# Generate image features

In [6]:
from feature_extraction import ImageFeatureExtractor

feat_extractor = ImageFeatureExtractor()
feats = data.sample(300).progress_apply(feat_extractor.get_features_from_row, axis=1)

feats.to_parquet(DATADIR / 'feats/feats.parquet')
feats

100%|██████████| 300/300 [00:04<00:00, 65.04it/s]


,centroid_east,centroid_east_sift,centroid_north,centroid_north_sift,diffs,diffs_sift,east_mean,east_mean_sift,east_median,east_median_sift,east_std,east_std_sift,n_matches,n_matches_sift,north_mean,north_mean_sift,north_median,north_median_sift,north_std,north_std_sift
85136,3.502350,0.549915,-3.035076,-0.024012,"[[2.514782, 0.18122101], [2.5854416, 0.1505279...","[[2.5884628295898438, 0.08149909973144531], [2...",2.554108,2.541552,2.514784,2.526184,0.417452,0.260540,68.0,209.0,0.099576,0.099695,0.125618,0.089310,0.382298,0.198273
136290,11.506493,-4.708549,5.279766,10.103982,"[[-0.7249651, -9.4635315], [-1.0776825, -7.738...","[[-0.9353466033935547, -9.26251220703125], [-0...",-0.812991,-0.879662,-0.989824,-0.980808,0.872745,0.621621,30.0,42.0,-8.492871,-8.265205,-8.522358,-8.291311,0.578975,0.692401
18718,9.715427,12.468045,-6.613441,-16.606871,"[[-0.5070572, 1.0871878], [-0.42164373, 0.8759...","[[-0.4313673973083496, 1.0892868041992188], [-...",-0.497519,-0.210993,-0.505507,-0.467739,0.106437,2.239009,15.0,77.0,1.093544,1.971719,1.087188,1.097641,0.136555,7.003173
53342,-30.387306,-1.383670,-16.927246,1.378942,"[[2.5643845, -0.15822601], [2.265068, -0.11657...","[[2.038318157196045, -0.21012496948242188], [1...",2.374594,2.359653,2.352081,2.044128,0.129152,3.916250,5.0,69.0,-0.145499,0.164763,-0.151215,-0.147034,0.017300,1.899017
134565,-3.744358,-2.992631,-13.642441,-8.178249,"[[-5.8362656, 4.9800644], [-5.7536926, 5.02681...","[[-5.773556232452393, 4.898353576660156], [-5....",-5.711296,-5.795464,-5.771236,-5.830492,0.367364,0.257972,70.0,150.0,5.027755,4.956969,5.017820,4.982771,0.134373,0.278270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77443,-22.956024,-22.914345,-47.913315,-48.550995,"[[-0.08794403, -0.17336273]]","[[-0.0689544677734375, -0.0782928466796875], [...",-0.087944,-0.068954,-0.087944,-0.068954,0.000000,0.000000,1.0,2.0,-0.173363,-0.078293,-0.173363,-0.078293,0.000000,0.000000
140638,21.219818,-8.872806,4.535847,-1.254074,"[[-12.5319605, 0.08419418], [-9.438492, 0.4960...","[[-8.4356689453125, 0.22563934326171875], [-9....",-9.689640,-8.298582,-9.250535,-7.858992,2.034852,1.343261,24.0,104.0,0.678299,0.260595,0.282055,0.289589,1.525867,0.367199
142565,-3.118137,-12.099001,-5.204918,3.726559,"[[-1.0966587, -5.6590233], [-2.0241013, -5.427...","[[-1.5559654235839844, -6.0698089599609375], [...",-1.830390,-2.323397,-1.854620,-1.856125,0.181435,2.592484,30.0,26.0,-5.523483,-5.012283,-5.531181,-5.372444,0.271766,2.045214
5087,-4.293159,-7.377655,5.313087,-1.460923,"[[0.6632767, -0.13124084], [0.633049, -0.11841...","[[0.5746726989746094, -0.1460723876953125], [0...",0.793230,0.801978,0.776123,0.822594,0.107846,0.255909,29.0,111.0,-0.143411,-0.125588,-0.141426,-0.112383,0.027741,0.288713


# Determine order of clips

In [8]:
from order_images import preprocess_df

train_ordered = preprocess_df(train, plot=False)
test_ordered = preprocess_df(test, plot=False)

train_ordered.to_parquet(DATADIR / 'processed/train_ordered.parquet')
test_ordered.to_parquet(DATADIR / 'processed/test_ordered.parquet')

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


KeyboardInterrupt: 

# Add lag/leap features

In [ ]:
from feature_extraction import add_lags

n_lags = 7

train_with_lags = add_lags(train, n_lags=n_lags)
test_with_lags = add_lags(test, n_lags=n_lags)

# Train model

In [ ]:
PERFORM_CV = False

# Predict for submission